# Hidden Markov Models


## Overview

The goal is to demonstrate the power of probabalistic models. We will build a word recognizer for American Sign Language (ASL) video sequences. In particular, this project employs hidden Markov models (HMM's) to analyze a series of measurements taken from videos of American Sign Language (ASL) collected for research (see the [RWTH-BOSTON-104 Database](http://www-i6.informatik.rwth-aachen.de/~dreuw/database-rwth-boston-104.php)).

In each video, an ASL signer is signing a meaningful sentence. In a typical ASL recognition system, you observe the XY coordinates of the speaker's left hand, right hand, and nose for every frame. The following diagram shows how the positions of the left hand (Red), right hand (Blue), and nose (Green) change over time in video number #66. Saturation of colors represents time elapsed.

<img src="./demo/hands_nose_position.png" alt="hands nose position">

This project will only use the Y-coordinates of each hand to construct the HMM. In Part 1 we will build a one dimensional model, recognizing words based only on a series of right-hand Y coordinates; in Part 2 we will go multidimensional and utilize both hands.

The words will be recognizing are "BUY", "HOUSE", and "CAR". These individual signs can be seen in the sign phrases from the dataset:

![](demo/buy_house_slow.gif) 

<p style="text-align:center; font-weight:bold"> JOHN CAN BUY HOUSE </p> 

![](demo/buy_car_slow.gif) 

<p style="text-align:center;  font-weight:bold"> JOHN BUY CAR [FUTURE] </p> 

### Encoding the HMM

Determine following values for each word:
1. the transition probabilities of each state
2. the mean & standard deviation of emission Gaussian distribution of each state



#### When calculating the self transition probability (e.g. P(B2 -> B2)), calculate it as 1 - exit transition probability (e.g. 1 - P(B2->B3))

Those values can be hardcoded in your program. Don't use round() from python.

Word | Frames | Observed sequence | Initial State1 | Initial State2 | Initial State3
--- | --- | --- | --- | --- | --- 
BUY | 6 | 36, 44, 52, 56, 49, 44 | 36, 44 | 52, 56 | 49, 44
BUY | 8 | 42, 46, 54, 62, 68, 65, 60, 56 | 42, 46, 54 | 62, 68, 65 | 60, 56
BUY | 10 | 42, 40, 41, 43, 52, 55, 59, 60, 55, 47 | 42, 40, 41|43, 52, 55|59, 60, 55, 47
CAR | 10 | 47, 39, 32, 34, 36, 42, 42, 42, 34, 25|47, 39, 32|34, 36, 42|42, 42, 34, 25
CAR | 9 | 35, 35, 43, 46, 52, 52, 56, 49, 45|35, 35, 43|46, 52, 52|56, 49, 45
CAR | 8 | 28, 35, 46, 46, 48, 43, 43, 40|28, 35, 46|46, 48, 43|43, 40
HOUSE| 15 | 37, 36, 32, 26, 26, 25, 23, 22, 21, 39, 48, 60, 70, 74, 77|37, 36, 32, 26, 26|25, 23, 22, 21, 39|48, 60, 70, 74, 77
HOUSE| 15 | 50, 50, 49, 47, 39, 39, 38, 38, 50, 56, 61, 67, 67, 67, 67|50, 50, 49, 47, 39|39, 38, 38, 50, 56|61, 67, 67, 67, 67
HOUSE| 16 | 45, 43, 44, 43, 40, 35, 36, 37, 39, 45, 60, 68, 66, 72, 72, 75|45, 43, 44, 43, 40|35, 36, 37, 39, 45|60, 68, 66, 72, 72, 75

As shown in the diagram below, each one of the three words (BUY, CAR, and HOUSE) has exactly **THREE hidden states** in its HMM. All words have equal probability of occuring. Modify the prior accordingly. All words must start from State 1 and can only transit to the next state or stay in the current one as shown below:

<img src="part_1_a_probs.png">

In [1]:
import hmm_submission_test as tests

Test on Linux/OS X system


In [2]:
import numpy as np
import operator

In [6]:
def HMM_encoder():

    """Word BUY"""
    b_prior_probs = {
        'B1': 0.333,
        'B2': 0.000,
        'B3': 0.000,
        'Bend': 0.000,
    }
    b_transition_probs = {
        'B1': {'B1': 0.625, 'B2': 0.375, 'B3': 0.000, 'Bend': 0.000},
        'B2': {'B1': 0.000, 'B2': 0.625, 'B3': 0.375, 'Bend': 0.000},
        'B3': {'B1': 0.000, 'B2': 0.000, 'B3': 0.625, 'Bend': 0.375},
        'Bend': {'B1': 0.000, 'B2': 0.000, 'B3': 0.000, 'Bend': 1.000},
    }
    # Parameters for end state is not required
    b_emission_paras = {
        'B1': (41.750, 2.773),
        'B2': (58.625, 5.678),
        'B3': (53.125, 5.418),
        'Bend': (None, None)
    }

    """Word CAR"""
    c_prior_probs = {
        'C1': 0.333,
        'C2': 0.000,
        'C3': 0.000,
        'Cend': 0.000,
    }
    c_transition_probs = {
        'C1': {'C1': 0.667, 'C2': 0.333, 'C3': 0.000, 'Cend': 0.000},
        'C2': {'C1': 0.000, 'C2': 0.000, 'C3': 1.000, 'Cend': 0.00},
        'C3': {'C1': 0.000, 'C2': 0.000, 'C3': 0.800, 'Cend': 0.200},
        'Cend': {'C1': 0.000, 'C2': 0.000, 'C3': 0.000, 'Cend': 1.000},
    }
    # Parameters for end state is not required
    c_emission_paras = {
        'C1': (35.667, 4.899),
        'C2': (43.667, 1.700),
        'C3': (44.200, 7.341),
        'Cend': (None, None)
    }

    """Word HOUSE"""
    h_prior_probs = {
        'H1': 0.333,
        'H2': 0.000,
        'H3': 0.000,
        'Hend': 0.000,
    }
    # Probability of a state changing to another state.
    h_transition_probs = {
        'H1': {'H1': 0.667, 'H2': 0.333, 'H3': 0.000, 'Hend': 0.000},
        'H2': {'H1': 0.000, 'H2': 0.857, 'H3': 0.143, 'Hend': 0.000},
        'H3': {'H1': 0.000, 'H2': 0.000, 'H3': 0.812, 'Hend': 0.188},
        'Hend': {'H1': 0.000, 'H2': 0.000, 'H3': 0.000, 'Hend': 1.000},
    }
    # Parameters for end state is not required
    h_emission_paras = {
        'H1': (45.333, 3.972),
        'H2': (34.952, 8.127),
        'H3': (67.438, 5.733),
        'Hend': (None, None)
    }

    return (b_prior_probs, b_transition_probs, b_emission_paras,
            c_prior_probs, c_transition_probs, c_emission_paras,
            h_prior_probs, h_transition_probs, h_emission_paras,)


### Creating the Viterbi Trellis

The goal here will be to use the HMM derived from above(states, prior probabilities, transition probabilities, and parameters of emission distribution) to build a viterbi trellis.  When provided with an evidence vector (list of observed right-hand Y coordinates), the function will return the most likely sequence of states that generated the evidence and the probabilty of that sequence being correct.

For example, an evidence vector [36, 44, 52, 53, 49, 44] should output a sequence ['B1', ... 'B2', ... 'B3']

If no sequence can be found, the algorithm should return one of the following tuples:
`(None, 0)` (null),  `([], 0)` (empty list) or  `(['C1', 'C1', ... 'C1'],0)` (Or all being the first state of that letter)

"No sequence can be found" means the probability reaches 0 midway. If you find an incomplete sequence with some probability, output that sequence with its probability. 

In [7]:
def gaussian_prob(x, para_tuple):
    
    if list(para_tuple) == [None, None]:
        return 0.0

    mean, std = para_tuple
    gaussian_percentile = (2 * np.pi * std**2)**-0.5 * \
                          np.exp(-(x - mean)**2 / (2 * std**2))
    return gaussian_percentile


In [9]:
def viterbi(evidence_vector, states, prior_probs,
            transition_probs, emission_paras):
    sequence = []
    probability = 0.0
    
    dict_1 = {}
    dict_2 = {}
    path = {}

    if len(evidence_vector) == 0:
        return sequence, probability

    for state in states:
        dict_1[state] = gaussian_prob(evidence_vector[0], emission_paras[state])\
                        * prior_probs[state]
        path[state] = [state]
        if dict_1[state] > probability:
            probability = dict_1[state]
            sequence = path[state] 
        
    if len(evidence_vector)== 1:
        return sequence, probability
    
    updated_path = {}        
    for state in states:
        #print('flag')
        prob_ls = {}
        for pre_state in (transition_probs[state].keys()):
            probability = gaussian_prob(evidence_vector[1], emission_paras[state])\
                          * dict_1[pre_state] * transition_probs[pre_state][state]
            prob_ls[probability] =  pre_state
        probability = max(prob_ls)
        dict_2[state] = probability
        updated_path[state] = path[prob_ls[probability]] + [state]
    path = updated_path


    for i in range(len(evidence_vector)):
        if i == 0 or i ==1:
            continue
        updated_path = {}
        for state in states:
            prob_ls = {}
            for pre_state in (transition_probs[state].keys()):
                probability = gaussian_prob(evidence_vector[i], emission_paras[state])\
                              * dict_2[pre_state] * transition_probs[pre_state][state]
                prob_ls[probability] =  pre_state
            probability = max(prob_ls)
            dict_1[state] = probability
            updated_path[state] = path[prob_ls[probability]] + [state]
        path = updated_path
        dict_2 = dict_1
        dict_1 = {}
        
    probability = 0  
    for state in dict_2:
        if dict_2[state] > probability:
            probability = dict_2[state]
            sequence = path[state] 

    return sequence, probability


### Multidimensional Output Probabilities

We have used right-hand Y-axis coordinates as our sole feature in the above part, now we are going to use both hands. Since sign language is two-handed, using both hands as features can increase the accuracy of our model when dealing with more complex sentences.

Here given with the transition probabilities, and the means & standard deviations for emission probabilties of left-hand Y-axis locations, following the same procedure conducted before.

One thing to notice is, the `viterbi` function is tested against single words. In other words, the input evidence vector will not transit between different words. However, for this step, the input evidence vector can be either a single word, or a verb phrase such as "BUY CAR" and "BUY HOUSE". Adjust the probabilities in the image below to adapt to this fact. Note that consecutive words should be different.

<img src="part_2_a_probs.png" alt="2a_probs">

BUY | State 1 | State 2 | State 3
--- | --- | --- | --- 
Mean | 108.200 | 78.670 | 64.182
Std | 17.314 | 1.886 | 5.573

CAR | State 1 | State 2 | State 3
--- | --- | --- | --- 
Mean | 56.300 | 37.110 | 50.000
Std | 10.659 | 4.306 | 7.826

HOUSE | State 1 | State 2 | State 3
--- | --- | --- | --- 
Mean | 53.600 | 37.168 | 74.176
Std | 7.392 | 8.875 | 8.347


In [11]:
def prob_HMM():

    b_prior_probs = {
        'B1': 0.333,
        'B2': 0.000,
        'B3': 0.000,
        'Bend': 0.000,
    }
    # example: {'B1': {'B1' : (right-hand Y, left-hand Y), ... }
    b_transition_probs = {
        'B1': {'B1': (0.625, 0.700), 'B2': (0.375, 0.300), 'B3': (0.000, 0.), 'Bend': (0.000, 0.000)},
        'B2': {'B1': (0., 0.), 'B2': (0.625, 0.05), 'B3': (0.375, 0.95), 'Bend': (0.000, 0.000)},
        'B3': {'B1': (0., 0.), 'B2': (0., 0.), 'B3': (0.625, 0.727), 'Bend': (0.125, 0.091), 'C1': (0.125, 0.091), 'H1': (0.125, 0.091)},
        'Bend': {'B1': (0., 0.), 'B2': (0., 0.), 'B3': (0., 0.), 'Bend': (1., 1.)},
    }
    # example: {'B1': [(right-mean, right-std), (left-mean, left-std)] ...}
    b_emission_paras = {
        'B1': [(41.750, 2.773), (108.200, 17.314)],
        'B2': [(58.625, 5.678), (78.670, 1.886)],
        'B3': [(53.125, 5.418), (64.182, 5.573)],
        'Bend': [(None, None), (None, None)]
    }

    """Word Car"""
    c_prior_probs = {
        'C1': 0.333,
        'C2': 0.000,
        'C3': 0.000,
        'Cend': 0.000,
    }
    c_transition_probs = {
        'C1': {'C1': (0.667, 0.700), 'C2': (0.333, 0.300), 'C3': (0.000, 0.000), 'Cend': (0.000, 0.000)},
        'C2': {'C1': (0.000, 0.000), 'C2': (0.000, 0.625), 'C3': (1.00, 0.375), 'Cend': (0.000, 0.000)},
        'C3': {'C1': (0.000, 0.000), 'C2': (0.000, 0.000), 'C3': (0.800, 0.625), 'Cend': (0.067, 0.125), 'B1': (0.067, 0.125), 'H1': (0.067, 0.125)},
        'Cend': {'C1': (0.000, 0.000), 'C2': (0.000, 0.000), 'C3': (0.000, 0.000), 'Cend': (1.000, 1.000)},
    }
    c_emission_paras = {
        'C1': [(35.667, 4.899), (56.3, 10.659)],
        'C2': [(43.667, 1.700), (37.11, 4.306)],
        'C3': [(44.200, 7.341), (50.000, 7.826)],
        'Cend': [(None, None), (None, None)]
    }

    """Word HOUSE"""
    h_prior_probs = {
        'H1': 0.333,
        'H2': 0.000,
        'H3': 0.000,
        'Hend': 0.000,
    }
    h_transition_probs = {
        'H1': {'H1': (0.667, 0.7), 'H2': (0.333, 0.300), 'H3': (0.000, 0.000), 'Hend': (0.000, 0.000)},
        'H2': {'H1': (0.000, 0.000), 'H2': (0.857, 0.842), 'H3': (0.143, 0.158), 'Hend': (0.000, 0.000)},
        'H3': {'H1': (0.000, 0.000), 'H2': (0.000, 0.000), 'H3': (0.812, 0.824), 'Hend': ((0.063, 0.059)), 'B1': (0.063, 0.059), 'C1': (0.063, 0.059)},
        'Hend': {'H1': (0.000, 0.000), 'H2': (0.000, 0.000), 'H3': (0.000, 0.000), 'Hend': (1.000, 1.000)},
    }
    h_emission_paras = {
        'H1': [(45.333, 3.972), (53.600, 7.392)],
        'H2': [(34.952, 8.127), (37.168, 8.875)],
        'H3': [(67.438, 5.733), (74.176, 8.347)],
        'Hend': [(None, None), (None, None)]
    }

    return (b_prior_probs, b_transition_probs, b_emission_paras,
            c_prior_probs, c_transition_probs, c_emission_paras,
            h_prior_probs, h_transition_probs, h_emission_paras,)


### Improving the Viterbi Trellis

Modify the Viterbi Trellis function to allow multiple observed values (Y location of right and left hands) for a state. 

In [12]:
def multidimensional_viterbi(evidence_vector, states, prior_probs,
                             transition_probs, emission_paras):
 
    sequence = []
    probability = 0.0
    
    dict_1 = {}
    dict_2 = {}
    path = {}
    updated_path = {}

    if len(evidence_vector) == 0:
        return sequence, probability

    for state in states:
        dict_1[state] = gaussian_prob(evidence_vector[0][0], emission_paras[state][0]) \
                 * gaussian_prob(evidence_vector[0][1], emission_paras[state][1]) \
                 * prior_probs[state]
        path[state] = [state]
        if dict_1[state] > probability:
            probability = dict_1[state]
            sequence = path[state]

    if len(evidence_vector)== 1:
        return sequence, probability
    
    for state in states:
        prob_ls = {}
        if state[0] == 'B':
            pre_states = ['B1', 'B2', 'B3', 'Bend']
        elif state[0] == 'C':
            pre_states = ['C1', 'C2', 'C3', 'Cend']
        elif state[0] == 'H':
            pre_states = ['H1', 'H2', 'H3', 'Hend']
            
        if state == 'B1':
            pre_states = ['B1', 'B2', 'B3', 'Bend', 'C3', 'H3']
        elif state == 'C1':
            pre_states = ['C1', 'C2', 'C3', 'Cend', 'B3', 'H3']
        elif state == 'H1':
            pre_states = ['H1', 'H2', 'H3', 'Hend', 'B3', 'C3']

        for pre_state in pre_states:
            probability = transition_probs[pre_state][state][0]\
                 * gaussian_prob(evidence_vector[1][0], emission_paras[state][0])\
                 * transition_probs[pre_state][state][1] \
                 * gaussian_prob(evidence_vector[1][1], emission_paras[state][1])\
                 * dict_1[pre_state]
            prob_ls[probability] =  pre_state
        probability = max(prob_ls)
        dict_2[state] = probability
        updated_path[state] = path[prob_ls[probability]] + [state]
    path = updated_path
    

    for i in range(len(evidence_vector)):
        if i == 0 or i == 1:
            continue
        updated_path = {}
        for state in states:
            prob_ls = {}
            if state[0] == 'B':
                pre_states = ['B1', 'B2', 'B3', 'Bend']
            elif state[0] == 'C':
                pre_states = ['C1', 'C2', 'C3', 'Cend']
            elif state[0] == 'H':
                pre_states = ['H1', 'H2', 'H3', 'Hend']
            
            if state == 'B1':
                pre_states = ['B1', 'B2', 'B3', 'Bend', 'C3', 'H3']
            elif state == 'C1':
                pre_states = ['C1', 'C2', 'C3', 'Cend', 'B3', 'H3']
            elif state == 'H1':
                pre_states = ['H1', 'H2', 'H3', 'Hend', 'B3', 'C3']

            for pre_state in pre_states:
                probability = transition_probs[pre_state][state][0]\
                     * gaussian_prob(evidence_vector[i][0], emission_paras[state][0])\
                     * transition_probs[pre_state][state][1] \
                     * gaussian_prob(evidence_vector[i][1], emission_paras[state][1])\
                     * dict_2[pre_state]
                prob_ls[probability] =  pre_state
            probability = max(prob_ls)
            dict_1[state] = probability
            updated_path[state] = path[prob_ls[probability]] + [state]
        path = updated_path
        dict_2 = dict_1
        dict_1 = {}

    probability = 0  
    for state in dict_2:
        if dict_2[state] > probability:
            probability = dict_2[state]
            sequence = path[state] 

    return sequence, probability
